# 美团用户评论情感分析

可以看出，使用SMOTE插值与简单的数据复制比起来，AUC率略有提高，实际预测效果也挺好

## 1. 数据读入

In [14]:
import pandas as pd
from matplotlib import pyplot as plt
import jieba
from sklearn.model_selection import train_test_split

In [3]:
data = pd.read_excel('meituan_preprocess.xlsx')
meituan = data.copy()

In [4]:
meituan['评论用户内容_分词'] = meituan['评论用户内容'].apply(lambda x: ' '.join(jieba.cut(x)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\62491\AppData\Local\Temp\jieba.cache
Loading model cost 1.055 seconds.
Prefix dict has been built successfully.


In [6]:
meituan.head()

,评论店铺,评论用户姓名,评论用户id,评论用户星级,评论用户菜品,评论用户内容,评论年份,评论月份,评论星期,评论小时,评论长度,评论用户内容_分词
0,158743609,养了四条鱼,156107390,50,男票甜蜜双人餐，提供免费WiFi,「 # 套餐 ： 男票 甜蜜 双人 餐 」 今天 休息 ， 团 了 个 男票 的 火锅 双人...,2021,9,5,21,113,「 # 套餐 ： 男票 甜蜜 双人 餐 」 今天 休...
1,158743609,养了四条鱼,156107390,50,真爱虾滑1份，包间免费,「 # 套餐 ： 真爱 虾 滑 」 好久没 吃火锅 了 ， 也 没 逛 江汉路 了 ， 嗯 ...,2021,9,5,21,118,「 # 套餐 ： 真 爱 虾 滑 」 好久没 吃火锅 ...
2,158743609,ddddddeft,1792254311,45,男票经典双人餐，提供免费WiFi,【 口味 】 味道 真的 不错 ， 价格 也 很 实惠 。 朋友 推荐 去 吃 的 。 今天...,2021,9,4,10,117,【 口味 】 味道 真的 不错 ， 价格 也 很 实...
3,158743609,Yoogut,1479967201,45,50元代金券1张，可叠加3张,本来 和 朋友 一起 去 吃 自助 的 ， 不 知道 怎么 莫名其妙 跑 去 女人 大 世界...,2021,8,6,23,118,本来 和 朋友 一起 去 吃 自助 的 ， 不 知道...
4,158743609,浪诗的浪潮,1954780805,50,男票经典双人餐，提供免费WiFi,「 # 套餐 ： 男票 经典 双人 餐 」 七夕 出来 吃火锅 ， 团购 的 券 刚好 用 ...,2021,8,1,18,143,「 # 套餐 ： 男票 经典 双人 餐 」 七夕 出...


## 2. 构建标签值

美团用户评论的评论星级为 0~5 星级，用 **评论用户星级** 这一属性代表，它的取值范围为 5-50，每 5 个单位代表 半颗星，例如 取值为 5 代表 0.5 星，取值 45 代表 4.5 星。

因此，我们将 3 星以上的取标签为 好评，3 星以及以下的为 差评，好评用 1 代表， 差评用 0 代表，我们的情感评分可以转化为标签值为 1 的概率值，这样我们就把情感分析问题转为文本分类问题了。

In [11]:
def label(score):
    if score > 30:
        return 1
    elif score <= 30:
        return 0
    
#标签转化
meituan['分类得分'] = meituan['评论用户星级'].map(lambda x: label(x))

## 3. 文本特征处理

中文文本特征处理，需要进行中文分词，分词之后需要过滤停用词，也就会去除无效的语气词之类的，最后要进行文本转向量，有词库表示法、TF-IDF、word2vec等。

首先对数据进行分割为测试集与训练集

In [15]:
##切分测试集、训练集
x_train, x_test, y_train, y_test = train_test_split(meituan['评论用户内容'], meituan['分类得分'], random_state=3, test_size=0.25)

接着引入停用词，去除文本中无效词

In [17]:
infile = open("stopwords.txt",encoding='utf-8')
stopwords_lst = infile.readlines()
stopwords = [x.strip() for x in stopwords_lst]

接着进行中文分词处理

In [19]:
def fenci(train_data):
    words_df = train_data.apply(lambda x:' '.join(jieba.cut(x)))
    return words_df

x_train_fenci = fenci(x_train)
x_train_fenci[:5]

13654     菜品   品种   比较   齐全   ，   口味   还   行   。   价格   稍贵
16311    「   #   套餐   ：   2   -   3   人网   红   火锅   套餐 ...
15304    价格   实惠   ，   口味   不错   ，   两个   人   团购   的   ...
10413    挺不错   的   ，   ，   口味   挺   好   ！   就是   每串   太...
10666    味道   不错   ，   就   在   家门口   ，   家   楼下   ，   好...
Name: 评论用户内容, dtype: object

最后就是文本特征提取

CountVectorizer旨在通过计数来将一个文档转换为向量。当不存在先验字典时，Countvectorizer作为Estimator提取词汇进行训练，并生成一个CountVectorizerModel用于存储相应的词汇向量空间。该模型产生文档关于词语的稀疏表示。下面举一个例子示范：

In [20]:
#用于转词向量的语料
yuliao = ['dog cat fish dog dog dog','cat eat fish','i like eat fish']

#sklearn库CountVectorizer转词向量
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
vector = cv.fit_transform(yuliao)
vector.todense()

matrix([[1, 4, 0, 1, 0],
        [1, 0, 1, 1, 0],
        [0, 0, 1, 1, 1]], dtype=int64)

In [21]:
cv.vocabulary_

{'dog': 1, 'cat': 0, 'fish': 3, 'eat': 2, 'like': 4}

In [25]:
#使用TF-IDF进行文本转向量处理
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv.fit(x_train_fenci)

d:\python\lib\site-packages\sklearn\feature_extraction\text.py:391: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'lex', 'll', 'mon', 'null', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TfidfVectorizer(max_features=3000, ngram_range=(1, 2),
                stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
                            '+', ',', '-', '--', '.', '..', '...', '......',
                            '...................', './', '.一', '记者', '数', '年',
                            '月', '日', '时', '分', '秒', '/', ...])

### 机器学习建模

特征和标签已经准备好了，接下来就是建模了。这里我们使用文本分类的经典算法朴素贝叶斯算法，而且朴素贝叶斯算法的计算量较少。特征值是评论文本经过TF-IDF处理的向量，标签值评论的分类共两类，好评是1，差评是0。情感评分为分类器预测分类1的概率值。

In [33]:
#计算分类效果的准确率
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, f1_score
classifier = MultinomialNB()
classifier.fit(tv.transform(x_train), y_train)
classifier.score(tv.transform(x_test), y_test)

0.8214616096207216

In [34]:
#计算分类器的AUC值
y_pred = classifier.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred)

0.8110449531563964

In [28]:
#计算一条评论文本的情感评分
def ceshi(model,strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(model.predict_proba(tv.transform(strings_fenci))[:,1])

In [36]:
#从美团网找两条评论来测试一下
test1 = '菜品种类超级多，熟食很多，水果饮料鲜榨种类很丰富，环境也蛮好的，就是进去的时候门不大好找！肉类品种很丰富，还有最爱的小龙虾，就是刚上五分钟就抢没了～盘子收的很勤快，服务态度也好，还有工作人员让我提意见，哈哈，我这个憨憨直说水果不甜...' #5星好评
test2 = '这不是欺骗顾客吗，根本没有龙虾，更气的是每个服务员都说有，让人白等！ ！欺骗顾客！没有就应该早点讲！ ！' #1星差评
print('好评实例的模型预测情感得分为{}\n差评实例的模型预测情感得分为{}'.format(ceshi(classifier,test1),ceshi(classifier,test2)))

好评实例的模型预测情感得分为0.6808873012465957
差评实例的模型预测情感得分为0.42542752977565185


可以看出，准确率和AUC值都非常不错的样子，但点评网上的实际测试中，5星好评模型预测出来了，1星差评缺预测错误。为什么呢？我们查看一下混淆矩阵

In [37]:
from sklearn.metrics import confusion_matrix
y_predict = classifier.predict(tv.transform(x_test))
cm = confusion_matrix(y_test, y_predict)
cm

array([[ 467,  784],
       [ 181, 3973]], dtype=int64)

可以看出，负类的预测非常不准，433单准确预测为负类的只有15.7%，应该是由于数据不平衡导致的，模型的默认阈值为输出值的中位数。比如逻辑回归的输出范围为[0,1]，当某个样本的输出大于0.5就会被划分为正例，反之为反例。在数据的类别不平衡时，采用默认的分类阈值可能会导致输出全部为正例，产生虚假的高准确度，导致分类失败。

处理样本不均衡问题的方法，首先可以选择调整阈值，使得模型对于较少的类别更为敏感，或者选择合适的评估标准，比如ROC或者F1，而不是准确度（accuracy）。另外一种方法就是通过采样（sampling）来调整数据的不平衡。其中欠采样抛弃了大部分正例数据，从而弱化了其影响，可能会造成偏差很大的模型，同时，数据总是宝贵的，抛弃数据是很奢侈的。另外一种是过采样，下面我们就使用过采样方法来调整。

### 过采样
单纯的重复了反例，因此会过分强调已有的反例。如果其中部分点标记错误或者是噪音，那么错误也容易被成倍的放大。因此最大的风险就是对反例过拟合。

In [38]:
meituan['分类得分'].value_counts()

1    16510
0     5108
Name: 分类得分, dtype: int64

In [39]:
#把0类样本复制10次，构造训练集
index_tmp = y_train==0
y_tmp = y_train[index_tmp]
x_tmp = x_train[index_tmp]
x_train2 = pd.concat([x_train,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp,x_tmp])
y_train2 = pd.concat([y_train,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp,y_tmp])

#使用过采样样本(简单复制)进行模型训练，并查看准确率
clf2 = MultinomialNB()
clf2.fit(tv.transform(x_train2), y_train2)
y_pred2 = clf2.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred2)

In [41]:
#查看此时的混淆矩阵
y_predict2 = clf2.predict(tv.transform(x_test))
cm = confusion_matrix(y_test, y_predict2)
cm

array([[1157,   94],
       [2486, 1668]], dtype=int64)

可以看出，即使是简单粗暴的复制样本来处理样本不平衡问题，负样本的识别率大幅上升了，变为77%，满满的幸福感呀~我们自己写两句评语来看看

In [43]:
ceshi(clf2,'排队人太多，环境不好，口味一般')

0.015253417587460566

可以看出把0类别的识别出来了，太棒了~

## 过采样（SMOTE算法）
SMOTE（Synthetic minoritye over-sampling technique,SMOTE），是在局部区域通过K-近邻生成了新的反例。相较于简单的过采样，SMOTE降低了过拟合风险，但同时运算开销加大。

In [48]:
#使用SMOTE进行样本过采样处理
from imblearn.over_sampling import SMOTE
oversampler= SMOTE(random_state=42)
x_train_vec = tv.transform(x_train)
x_resampled, y_resampled = oversampler.fit_resample(x_train_vec, y_train)

In [49]:
#原始的样本分布
y_train.value_counts()

1    12356
0     3857
Name: 分类得分, dtype: int64

In [50]:
#经过SMOTE算法过采样后的样本分布情况
pd.Series(y_resampled).value_counts()

0    12356
1    12356
Name: 分类得分, dtype: int64


我们经过插值，把0类数据也丰富为 12356 个数据了，这时候正负样本的比例为1:1，接下来我们用平衡后的数据进行训练，效果如何呢，好期待啊~

In [51]:
#使用过采样样本(SMOTE)进行模型训练，并查看准确率
clf3 = MultinomialNB()
clf3.fit(x_resampled, y_resampled)
y_pred3 = clf3.predict_proba(tv.transform(x_test))[:,1]
roc_auc_score(y_test,y_pred3)

0.8152047452072044

In [52]:
#查看此时的准确率
y_predict3 = clf3.predict(tv.transform(x_test))
cm = confusion_matrix(y_test, y_predict3)
cm

array([[ 882,  369],
       [1050, 3104]], dtype=int64)

In [53]:
#到网上找一条差评来测试一下情感评分的预测效果
test3 = '这不是欺骗顾客吗，根本没有龙虾，更气的是每个服务员都说有，让人白等！ ！欺骗顾客！没有就应该早点讲！ ！'
ceshi(clf3,test3)

0.25750452237572696

可以看出，使用SMOTE插值与简单的数据复制比起来，AUC率略有提高，实际预测效果也挺好

In [55]:
#词向量训练
tv2 = TfidfVectorizer(stop_words=stopwords, max_features=3000, ngram_range=(1,2))
tv2.fit(meituan['评论用户内容'])

#SMOTE插值
X_tmp = tv2.transform(meituan['评论用户内容'])
y_tmp = meituan['分类得分']
sm = SMOTE(random_state=0)
X,y = sm.fit_resample(X_tmp, y_tmp)

clf = MultinomialNB()
clf.fit(X, y)

def fenxi(strings):
    strings_fenci = fenci(pd.Series([strings]))
    return float(clf.predict_proba(tv2.transform(strings_fenci))[:,1])

In [57]:
fenxi('不明白那么多五星好评是怎么来的，价格贵你好吃就算了，重点是贵还难吃，两个人吃了300多，麻辣锅底不麻不辣，一股中药味，番茄锅没有蕃茄味，只有酸酸的还不是番茄的味道，69的肥牛就10片，网红企鹅真的巨难吃，除了跳水啥也不是，唯一能过得去的只有去骨鸡肉，莴笋还可以，就是特别大的一块，都不知道怎么下口吃，跑这么远吃的一点都不开心，买单的时候也没有服务员，居然有人说服务比海底捞好？？？请的海底捞的黑粉吗？')

0.06961621359910641